# Otázky

1. když provedu `single_correction`, mám vrátit tu korekci a pak teprve
 updatovat hyperparametry? Dává to smysl, jinak by vyšlo několikrát to samé...

In [73]:
import numpy as np
import common
from exp_family import update_IW, init_IW_hyp, get_IW_pars_from_hyp
import exp_family
from typing import Tuple
from collections import defaultdict

In [74]:
Matrix = np.ndarray
MultiShape = np.ndarray
Scalar = np.number
Vector = np.ndarray

class IWPrior:
    hp: MultiShape = None
    p: Scalar = None
    
    def __init__(self, nu: Scalar, psi: Matrix):
        self.hp = np.array([
            -0.5 * psi, 
            -0.5 * (nu + psi.shape[0] + 1)
        ])
    
    @property
    def psi(self) -> Matrix:
        return -2 * self.hp[0]
    
    @property
    def nu(self) -> Scalar:
        return -2 * self.hp[1] - self.p - 1
    
    @property
    def p(self) -> int:
        return self.hp[0].shape[0]
    
    def expect(self):
        return self.psi / (self.nu - self.p - 1)
    
class MeasurementNode:
    P_prior: IWPrior = None
    R_prior: IWPrior = None
    F: Matrix = None
    Q: Matrix = None
    last_state: Vector = None
        
    logger = defaultdict(list)
    
    def __init__(self,
                 x: Vector,
                 P_prior: IWPrior,
                 R_prior: IWPrior,
                 F: Matrix,
                 Q: Matrix,
                 H: Matrix,
                 rho: Scalar,
                 tau: Scalar,
                 N = 10,
                ):
        self.last_state = x
        self.P_prior = P_prior
        self.R_prior = R_prior
        self.F = F
        self.Q = Q
        self.H = H
        self.rho = rho
        self.tau = tau
        
        self.P = self.P_prior.expect()
        
    def predict_state(self) -> Tuple[Vector, Matrix]:
        return common.time_update(self.last_state, self.P, self.F, self.Q)
    
    def _single_update(self, state_prediction, P_prediction, measurement,
                     init_hyp_P, init_hyp_R):
        x = state_prediction
        P = P_prediction
        for i in range(N):
            R, hyp_R = update_IW(init_hyp_R, 
                                 measurement, 
                                 self.H @ x, 
                                 self.H @ P @ self.H.T)
            P, hyp_P = update_IW(init_hyp_P, x, state_prediction, P)
            P, x = common.kalman_correction(H, P, R, state_prediction, measurement)
        return x, P, hyp_P, hyp_R
    
    def _init_hyp_P(self, P_predicted):
        init = exp_family.init_P_hyp(self.tau, P_predicted)
        return init
    
    
    def _init_hyp_R(self):
        init = exp_family.init_R_hyp(self.rho, self.R_prior.psi, self.R_prior.nu)
        return init
    
    
    def _single_kf(self, measurement):
        x_predicted, P_predicted = self.predict_state()
        init_hyp_P = self._init_hyp_P(P_predicted)
        init_hyp_R = self._init_hyp_R()
        return self._single_update(x_predicted, P_predicted, measurement,
                                  init_hyp_P, init_hyp_R)
    
    def single_kf(self, measurement):
        x, P, hyp_P, hyp_R = self._single_kf(measurement)
        self.last_state = x
        self.P = P
        self.P_prior.hp = hyp_P
        self.R_prior.hp = hyp_R
        
    def log(self, key, val):
        self.logger[key].append(val)

In [75]:
def node_factory(x, P, u, U, F, Q, H, rho, tau):
    P_p = IWPrior(P.shape[0] + tau + 1, tau * P)
    R_p = IWPrior(u, U)    
    return MeasurementNode(x, P_p, R_p, F, Q, H, rho, tau)

In [76]:
traj, xk, P, tau, rho, u, U, H, F, Q, N = common.init_all()

In [77]:
m1 = node_factory(xk, P, u, U, F, Q, H, rho, tau)
m2 = node_factory(xk, P, u, U, F, Q, H, rho, tau)

In [78]:
m1.predict_state()
m1.single_kf(traj.Y.T[0])
""

''

In [91]:
traj, xk, P, tau, rho, u, U, H, F, Q, N = common.init_all()

In [92]:
t = 0
x_log = []
P_log = []
for zk in traj.Y.T:
    #print('===== Step: ', t, '======') 
    t += 1
    m1 = node_factory(xk, P, u, U, F, Q, H, rho, tau)
    #### Time update
    # xkl
    xk = common.predict_state(F, xk)  # xll
    # Pkl
    P = common.predict_PECM(F, P, Q)  # Pll
    assert np.isclose(xk, m1.predict_state()[0]).all() 
    assert np.isclose(P, m1.predict_state()[1]).all().all()
    
    #### Measurement update
    ## Initialization - step 3
    # Pikk
    Pik = P
    # xikk
    xikk = xk
    
    # tkl, Tkl, ukl, Ukl 
    tkk, Tkk, ukl, Ukl = common.init(P, tau, P.shape[0], rho, u, U.shape[0], U)
    
    hyp_P = init_IW_hyp(Tkk, tkk)
    hyp_P_prev = hyp_P.copy()
    
    hyp_R = init_IW_hyp(Ukl, ukl)
    hyp_R_prev = hyp_R.copy()
    ## END of Initialization - step 3
    
    assert hyp_P[1] == m1._init_hyp_P(P)[1]
    assert np.isclose(hyp_P[0], m1._init_hyp_P(P)[0]).all().all()
    

    #assert hyp_R[1] == m1._init_hyp_R()[1]
    assert hyp_R[1] == m1._init_hyp_R()[1]
    assert np.isclose(hyp_R[0], m1._init_hyp_R()[0]).all().all()

    
    ## VB iters
    for i in range(N):
        Rjk, hyp_R = update_IW(hyp_R_prev, zk, H @ xikk, H @ Pik @ H.T)
        Pjkl, hyp_P = update_IW(hyp_P_prev, xikk, xk, Pik)
        # Pjkk, xjkk
        Pik, xikk = common.kalman_correction(H, Pjkl, Rjk, xk, zk)
    
    # Assignment - step 13
    # xkk
    xk = xikk
    # Pkk
    P = Pik
    # ukk, Ukk
    U, u = get_IW_pars_from_hyp(hyp_R)
    #P, p = get_IW_pars_from_hyp(hyp_P)
    x_, P_, *_ = m1._single_kf(zk)
    assert np.isclose(xk, x_).all()
    assert np.isclose(P, P_).all().all()
    
    x_log.append(xk)
    P_log.append(P)
    
x_log = np.array(x_log).squeeze().T
P_log = np.array(P_log).squeeze()

In [ ]:
t = 0
x_log = []
P_log = []
for zk in traj.Y.T:
    #print('===== Step: ', t, '======') 
    t += 1
    m1 = node_factory(xk, P, u, U, F, Q, H, rho, tau)
    #### Time update
    # xkl
    xk = common.predict_state(F, xk)  # xll
    # Pkl
    P = common.predict_PECM(F, P, Q)  # Pll
    assert np.isclose(xk, m1.predict_state()[0]).all() 
    assert np.isclose(P, m1.predict_state()[1]).all().all()
    
    #### Measurement update
    ## Initialization - step 3
    # Pikk
    Pik = P
    # xikk
    xikk = xk
    
    # tkl, Tkl, ukl, Ukl 
    tkk, Tkk, ukl, Ukl = common.init(P, tau, P.shape[0], rho, u, U.shape[0], U)
    
    hyp_P = init_IW_hyp(Tkk, tkk)
    hyp_P_prev = hyp_P.copy()
    
    hyp_R = init_IW_hyp(Ukl, ukl)
    hyp_R_prev = hyp_R.copy()
    ## END of Initialization - step 3
    
    assert hyp_P[1] == m1._init_hyp_P(P)[1]
    assert np.isclose(hyp_P[0], m1._init_hyp_P(P)[0]).all().all()
    

    #assert hyp_R[1] == m1._init_hyp_R()[1]
    assert hyp_R[1] == m1._init_hyp_R()[1]
    assert np.isclose(hyp_R[0], m1._init_hyp_R()[0]).all().all()

    
    ## VB iters
    for i in range(N):
        Rjk, hyp_R = update_IW(hyp_R_prev, zk, H @ xikk, H @ Pik @ H.T)
        Pjkl, hyp_P = update_IW(hyp_P_prev, xikk, xk, Pik)
        # Pjkk, xjkk
        Pik, xikk = common.kalman_correction(H, Pjkl, Rjk, xk, zk)
    
    # Assignment - step 13
    # xkk
    xk = xikk
    # Pkk
    P = Pik
    # ukk, Ukk
    U, u = get_IW_pars_from_hyp(hyp_R)
    #P, p = get_IW_pars_from_hyp(hyp_P)
    x_, P_, *_ = m1._single_kf(zk)
    assert np.isclose(xk, x_).all()
    assert np.isclose(P, P_).all().all()
    
    x_log.append(xk)
    P_log.append(P)
    
x_log = np.array(x_log).squeeze().T
P_log = np.array(P_log).squeeze()

In [90]:
# KF, just measurements
np.sqrt(((x_log[:2] - traj.X[:2]) ** 2 ).mean()), np.sqrt(((traj.Y[:2] - traj.X[:2]) ** 2 ).mean())

(0.4619606664283383, 0.7668096325216082)

In [85]:
# velocity: KF
np.sqrt(((x_log[2:] - traj.X[2:]) ** 2 ).mean())

1.246633480261624